### Install selenium


In [1]:
%pip install selenium

     ---------------------------------------- 0.0/58.3 kB ? eta -:--:--
     ---------------------------------------- 58.3/58.3 kB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.2 MB 2.2 MB/s eta 0:00:05
    --------------------------------------- 0.2/10.2 MB 1.6 MB/s eta 0:00:07
    --------------------------------------- 0.2/10.2 MB 1.4 MB/s eta 0:00:08
   - -------------------------------------- 0.3/10.2 MB 1.3 MB/s eta 0:00:08
   - -------------------------------------- 0.4/10.2 MB 1.6 MB/s eta 0:00:06
   -- ------------------------------------- 0.5/10.2 MB 1.8 MB/s eta 0:00:06
   -- ------------------------------------- 0.7/10.2 MB 1.9 MB/s eta 0:00:05
   --- ------------------------------------ 0.8/10.2 MB 2.0 MB/s eta 0:00:05
   --- ------------------------------------ 1.0/10.2 MB 2.1 MB/s eta 0:00:05
   ---- ----------------------------------- 1.1/10.2 MB 2.3 MB/s eta 0:00:04
   -----

### Get William hill data


In [11]:
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By


premLeague = "https://sports.williamhill.com/betting/en-gb/football/competitions/OB_TY295/English-Premier-League/matches/OB_MGMB/Match-Betting"

driver = webdriver.Firefox()
driver.get(premLeague)
matches = driver.find_elements(By.CSS_SELECTOR, ".sp-o-market__title > a")[0:9]
matches_links = []

# Get links for the matches
# [[match, link]]
for match in matches:
    matches_links.append([match.text, match.get_attribute('href')])

driver.close()

# For each link, we fetch the player data
# create an object [name, odds, match]
# details = [   name, match,
#              {"first":[first[0],first[1]]} ,
# {"anytime":[anytime[0],anytime[1]]},
# {"last":[last[0],last[1]]},
# {"two_or_more":[two_or_more[0],two_or_more[1]]},
# {"hattrick":[hattrick[0],hattrick[1]]},
# {"first_and_last":[first_and_last[0],first_and_last[1]]},
# {"first_or_last":[first_or_last[0],first_or_last[1]]}
# ]
details = []
for link in matches_links:  # Here you're getting data for each link .i.e each loop has one match
    driver = webdriver.Firefox()
    driver.get(link[1])
    players = driver.find_elements(
        By.CSS_SELECTOR, "article.cpt-goalscorer__player")

    if (len(players) == 0):  # check if the details are loaded. Otherwise, it is in the following GW
        continue

    players_details = []

    for index, tag in enumerate(players):  # for each match get player details
        # Get player name
        player = tag.find_element(
            By.TAG_NAME, 'h3').get_attribute("textContent")
        odds = tag.find_elements(By.CLASS_NAME, 'selectiondetails')

        def splitOdds(odd):
            if (len(odd.split('/')) > 1):
                return odd.split('/')
            else:
                return [odd]

        # Get players odds
        if (index+1 == len(players)):
            # For the no goal scored odd
            first = odds[0].get_attribute("data-odds")
            odds_details = {"name": player, "match": link[0], "first": first}

            players_details.append(odds_details)
            continue
        else:
            first = odds[0].get_attribute("data-odds")
            anytime = odds[1].get_attribute("data-odds")
            last = odds[2].get_attribute("data-odds")
            two_or_more = odds[3].get_attribute("data-odds")
            hattrick = odds[4].get_attribute("data-odds")
            first_and_last = odds[5].get_attribute("data-odds")
            first_or_last = odds[6].get_attribute("data-odds")

            odds_details = {
                "name": player,
                "match": link[0],
                "first": first,
                "anytime": anytime,
                "last": last,
                "two_or_more": two_or_more,
                "hattrick": hattrick,
                "first_and_last": first_and_last,
                "first_or_last": first_or_last
            }
            players_details.append(odds_details)

    # group the match details
    details.append(players_details)

    driver.close()


print(details)
print(matches_links)

[[{'name': 'Gabriel Jesus', 'match': 'Arsenal v Wolves', 'first': '9/2', 'anytime': '6/4', 'last': '9/2', 'two_or_more': '17/2', 'hattrick': '35/1', 'first_and_last': '14/1', 'first_or_last': '5/2'}, {'name': 'Bukayo Saka', 'match': 'Arsenal v Wolves', 'first': '9/2', 'anytime': '8/5', 'last': '9/2', 'two_or_more': '17/2', 'hattrick': '40/1', 'first_and_last': '14/1', 'first_or_last': '13/5'}, {'name': 'Eddie Nketiah', 'match': 'Arsenal v Wolves', 'first': '9/2', 'anytime': '6/4', 'last': '9/2', 'two_or_more': '8/1', 'hattrick': '35/1', 'first_and_last': '14/1', 'first_or_last': '5/2'}, {'name': 'Leandro Trossard', 'match': 'Arsenal v Wolves', 'first': '5/1', 'anytime': '9/5', 'last': '5/1', 'two_or_more': '10/1', 'hattrick': '45/1', 'first_and_last': '16/1', 'first_or_last': '3/1'}, {'name': 'Gabriel Martinelli', 'match': 'Arsenal v Wolves', 'first': '11/2', 'anytime': '7/4', 'last': '5/1', 'two_or_more': '10/1', 'hattrick': '45/1', 'first_and_last': '16/1', 'first_or_last': '29/10'},

In [13]:
details

[[{'name': 'Gabriel Jesus',
   'match': 'Arsenal v Wolves',
   'first': '9/2',
   'anytime': '6/4',
   'last': '9/2',
   'two_or_more': '17/2',
   'hattrick': '35/1',
   'first_and_last': '14/1',
   'first_or_last': '5/2'},
  {'name': 'Bukayo Saka',
   'match': 'Arsenal v Wolves',
   'first': '9/2',
   'anytime': '8/5',
   'last': '9/2',
   'two_or_more': '17/2',
   'hattrick': '40/1',
   'first_and_last': '14/1',
   'first_or_last': '13/5'},
  {'name': 'Eddie Nketiah',
   'match': 'Arsenal v Wolves',
   'first': '9/2',
   'anytime': '6/4',
   'last': '9/2',
   'two_or_more': '8/1',
   'hattrick': '35/1',
   'first_and_last': '14/1',
   'first_or_last': '5/2'},
  {'name': 'Leandro Trossard',
   'match': 'Arsenal v Wolves',
   'first': '5/1',
   'anytime': '9/5',
   'last': '5/1',
   'two_or_more': '10/1',
   'hattrick': '45/1',
   'first_and_last': '16/1',
   'first_or_last': '3/1'},
  {'name': 'Gabriel Martinelli',
   'match': 'Arsenal v Wolves',
   'first': '11/2',
   'anytime': '7/4

### Store Williamhill data to Excel sheet (remember to change GW)


In [14]:
import pandas as pd
# len(det)
init_data = {"name": [],
             "match": [],
             "first": [],
             "anytime": [],
             "last": [],
             "two_or_more": [],
             "hattrick": [],
             "first_and_last": [],
             "first_or_last": []}

df = pd.DataFrame(init_data)

for items in details:  # for each match in object
    for item in items:  # for each player in match
        df.loc[len(df)] = item
with pd.ExcelWriter("../data/scrapped/williamhill.xlsx", mode="a") as writer:
    df.to_excel(writer, sheet_name="GW14")
# df.to_excel("../data/scrapped/williamhill_v1.xlsx", sheet_name="GW12")

### Sporting Index


#### Getting Data

Go to [here](https://www.sportingindex.com/sports/en-GB/spread-betting/football/footballpopular/spr/c57)[right]

Enter the following command in the console:

- Command 1:

```js
var premLeague_ = document.getElementById("premier-league");
```

- Command 2:

```js
var premMatches = premLeague_.getElementsByClassName("cpi");
```

- Command 3:

```js
var details = [];
```

- Command 4:

````js
     for (let i=0; i<10;i++){
         details.push({match:premMatches[i].querySelector('.name').outerText,sell_buy:premMatches[i].querySelector('.cpi__price').outerText})
     }
 ```

 - Then copy the returned object and save it as sPDetails in the next block

````


In [1]:
# Returned Object from the console
SpDetails = [
    {
        "match": "Arsenal v Wolves",
        "sell_buy": "1.75 - 1.95"
    },
    {
        "match": "Brentford v Luton",
        "sell_buy": "1 - 1.2"
    },
    {
        "match": "Burnley v Sheff Utd",
        "sell_buy": "0.65 - 0.85"
    },
    {
        "match": "Nottingham Forest v Everton",
        "sell_buy": "0 - 0.2"
    },
    {
        "match": "Newcastle v Man Utd",
        "sell_buy": "0.35 - 0.55"
    },
    {
        "match": "Bournemouth v Aston Villa",
        "sell_buy": "0.3 - 0.5"
    },
    {
        "match": "Chelsea v Brighton",
        "sell_buy": "0.7 - 0.9"
    },
    {
        "match": "Liverpool v Fulham",
        "sell_buy": "1.75 - 1.95"
    },
    {
        "match": "West Ham v Crystal Palace",
        "sell_buy": "0.35 - 0.55"
    },
    {
        "match": "Man City v Tottenham",
        "sell_buy": "1.8 - 2"
    },
    {
        "match": "Arsenal v Wolves",
        "sell_buy": "1.7 - 1.9"
    },
    {
        "match": "Brentford v Luton",
        "sell_buy": "1.05 - 1.25"
    },
    {
        "match": "Burnley v Sheff Utd",
        "sell_buy": "1.6 - 1.8"
    },
    {
        "match": "Nottingham Forest v Everton",
        "sell_buy": "0 - 0.2"
    },
    {
        "match": "Newcastle v Man Utd",
        "sell_buy": "0.35 - 0.55"
    },
    {
        "match": "Bournemouth v Aston Villa",
        "sell_buy": "0.3 - 0.5"
    },
    {
        "match": "Chelsea v Brighton",
        "sell_buy": "0.7 - 0.9"
    },
    {
        "match": "Liverpool v Fulham",
        "sell_buy": "1.75 - 1.95"
    },
    {
        "match": "West Ham v Crystal Palace",
        "sell_buy": "0.35 - 0.55"
    },
    {
        "match": "Man City v Tottenham",
        "sell_buy": "1.8 - 2"
    }
]

SpDetails

[{'match': 'Arsenal v Wolves', 'sell_buy': '1.75 - 1.95'},
 {'match': 'Brentford v Luton', 'sell_buy': '1 - 1.2'},
 {'match': 'Burnley v Sheff Utd', 'sell_buy': '0.65 - 0.85'},
 {'match': 'Nottingham Forest v Everton', 'sell_buy': '0 - 0.2'},
 {'match': 'Newcastle v Man Utd', 'sell_buy': '0.35 - 0.55'},
 {'match': 'Bournemouth v Aston Villa', 'sell_buy': '0.3 - 0.5'},
 {'match': 'Chelsea v Brighton', 'sell_buy': '0.7 - 0.9'},
 {'match': 'Liverpool v Fulham', 'sell_buy': '1.75 - 1.95'},
 {'match': 'West Ham v Crystal Palace', 'sell_buy': '0.35 - 0.55'},
 {'match': 'Man City v Tottenham', 'sell_buy': '1.8 - 2'},
 {'match': 'Arsenal v Wolves', 'sell_buy': '1.7 - 1.9'},
 {'match': 'Brentford v Luton', 'sell_buy': '1.05 - 1.25'},
 {'match': 'Burnley v Sheff Utd', 'sell_buy': '1.6 - 1.8'},
 {'match': 'Nottingham Forest v Everton', 'sell_buy': '0 - 0.2'},
 {'match': 'Newcastle v Man Utd', 'sell_buy': '0.35 - 0.55'},
 {'match': 'Bournemouth v Aston Villa', 'sell_buy': '0.3 - 0.5'},
 {'match': 

#### Store Williamhill data to Excel sheet (remember to change game)


In [ ]:
jk = {'uie':123,'item':453}
for i in jk:


In [ ]:
import pandas as pd
# len(det)
init_data = {"match": [],
             "sell_buy": [],
             }

df = pd.DataFrame(init_data)

for item in SpDetails:  # for each match in object
    df.loc[len(df)] = item

df

with pd.ExcelWriter("../data/scrapped/sportingindex.xlsx", mode="a") as writer:
    df.to_excel(writer, sheet_name="GW14")
# df.to_excel("../data/scrapped/sportingindex.xlsx", sheet_name="GW14")